# AutoGluon: Object Detection on COCO Dataset

## Objective
This notebook demonstrates **object detection** using AutoGluon on COCO-format datasets. Object detection locates and classifies multiple objects within images.

## Use Case
Object detection is useful for:
- Autonomous vehicles (detecting pedestrians, vehicles, signs)
- Security and surveillance (person detection, intrusion detection)
- Retail analytics (product counting, shelf monitoring)
- Medical imaging (tumor detection, cell counting)
- Manufacturing (defect detection, part identification)

## COCO Dataset Format
The COCO format includes:
- Images with multiple objects
- Bounding box coordinates (x, y, width, height)
- Object categories/labels
- Annotations in JSON format

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil
import json

In [ ]:
# Load dataset
# TODO: Upload your COCO-format dataset or use URL
# COCO dataset should have:
# - Images directory
# - Annotations JSON file with format: {"images": [...], "annotations": [...], "categories": [...]}

# Example structure:
# /dataset/
#   /images/
#     image1.jpg
#     image2.jpg
#   annotations.json

# For AutoGluon, convert COCO to CSV format:
# CSV columns: 'image', 'rois' (bounding boxes), 'label'

# Example: train_data = TabularDataset('path/to/coco_train.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print(train_data.head())

In [ ]:
# Set label column
# For object detection, this is the object category
LABEL = 'label'  # TODO: Replace with your label column name

In [ ]:
# Auto-detect problem type based on label
# Object detection is a specialized classification task
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default for object detection
    problem_type = 'classification'
    eval_metric = 'roc_auc'  # Can also use mAP for object detection

print(f"Problem Type: {problem_type} (Object Detection)")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: Object detection identifies and localizes multiple objects per image.")

In [ ]:
# Train the model
# AutoGluon will use object detection models like YOLO, Faster R-CNN, etc.
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-object-detection-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900
)

print("Model training completed!")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions (Detected Objects):")
    print(predictions.head())
    
    # For object detection, predictions include:
    # - Bounding boxes (coordinates)
    # - Object classes
    # - Confidence scores
    
    if problem_type == 'classification':
        pred_probs = predictor.predict_proba(test_data)
        print("\nPrediction Probabilities:")
        print(pred_probs.head())
        
    # Example output format
    print("\nObject Detection Output Format:")
    print("Each prediction contains: [class, confidence, x1, y1, x2, y2]")
    print("Where (x1,y1) is top-left and (x2,y2) is bottom-right of bounding box")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-object-detection-model'
zip_filename = 'autogluon_object_detection_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")